# SD-TSIA 211 : TP 1

### Question 3.1

$$
f_1(w_0,w) = \frac{1}{n} \sum_{i=1}^{n} \log(1+\exp(-y_i(x_i^Tw+w_0))) + \frac{\rho}{2}  \left\|w\right\|_2^2
$$

Let's compute the gradient of $f_1$.

Let's define the sigmoide function.

$$
\sigma(a) = (1 + e^{-a})^{-1}
$$
$$
\sigma^{'}(a) = \sigma(a) (1-\sigma(a))
$$

$$
\nabla f_1(w_0,w) = \begin{pmatrix} 
    -\frac{1}{n} \sum_{i=1}^{n} y_i \sigma(-y_i(x_i^Tw+w_0)) \\
    -\frac{1}{n} \sum_{i=1}^{n} x_{1,i} y_i \sigma(-y_i(x_i^Tw+w_0)) + \rho w_1 \\
    \vdots \\
    -\frac{1}{n} \sum_{i=1}^{n} x_{p,i} y_i \sigma(-y_i(x_i^Tw+w_0)) + \rho w_p
    \end{pmatrix}
$$

Let's now compute the Hessian matrix of $f_1$.

Since $y_i^2 = 1$, we have :
$$
H_{f_1}^{(1,1)} = \frac{1}{n} \sum_{i=1}^{n} \sigma(-y_i(x_i^Tw+w_0))(1-\sigma(-y_i(x_i^Tw+w_0)))
$$
$\forall l \in [2,p+1],$
$$
H_{f_1}^{(l,1)} = \frac{1}{n} \sum_{i=1}^{n} x_{l,i} \sigma(-y_i(x_i^Tw+w_0))(1-\sigma(-y_i(x_i^Tw+w_0)))
$$
$\forall (l,k) \in [2,p+1]^2,$
$$
H_{f_1}^{(l,k)} = \frac{1}{n} \sum_{i=1}^{n} x_{l,i} x_{k,i} \sigma(-y_i(x_i^Tw+w_0))(1-\sigma(-y_i(x_i^Tw+w_0))) + \rho \delta _{l,k}
$$

### Question 3.2

In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.linalg import norm, solve
from scipy.optimize import check_grad
import matplotlib.pyplot as plt
import time

def load_data(file_name_matrix='tfidf_matrix.npz', file_name_feature_names='feature_names.npy',
	      file_name_labels='train_labels.npy', samples_in_train_set=10000,
	      samples_in_test_set=137562):
	# Recuperation des donnees
	TF_IDF_matrix = sp.load_npz(file_name_matrix)
	TF_IDF_feature_names = np.load(file_name_feature_names)
	train_labels = np.load(file_name_labels, allow_pickle=True)
	train_labels_numeric = ((train_labels == 'Oui') + 0)

	X = TF_IDF_matrix[:samples_in_train_set].toarray()
	y = train_labels_numeric[:samples_in_train_set] * 2 - 1

	X_test = TF_IDF_matrix[samples_in_train_set:samples_in_train_set+samples_in_test_set].toarray()
	y_test = train_labels_numeric[samples_in_train_set:samples_in_train_set+samples_in_test_set] * 2 - 1


	# Standardisation des données
	std_X = np.maximum(np.std(X, axis=0), 1e-7)
	X = X / std_X
	X_test = X_test / std_X

	n = X.shape[0]
	n_test = X_test.shape[0]
	m = X.shape[1]

	eX = X
	eX_test = X_test

	# Ajout d'une colonne de uns
	#eX = np.hstack((np.ones((n,1)), X))
	#eX_test = np.hstack((np.ones((n_test,1)), X_test))

	return eX, y, eX_test, y_test

In [2]:
def sigmoide(a) :
    return (1 / (1 + np.exp(-a)))
a, b, c, d = load_data('tfidf_matrix_97MB.npz', 'feature_names_97MB.npy', 'train_labels.npy')
x = a[0:30,0:30] #On réduit la dimension pour que ça ne soit pas trop long (et que mon ordi ne flambe pas)
y = b
n, p = x.shape
rho = 1/n
print(x)
print(y)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         4.18178779 0.         0.         5.2446769
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.  

In [3]:
print(n,p)

30 30


In [4]:
def deriv_f1(w0,w) :
    Grad = np.zeros(p+1)
    Hess = np.zeros((p+1,p+1))
    IntoSigma = np.zeros((p+1,n))

    Grad0 = 0
    for i in range(n) :
        IntoSigma[0][i] = np.dot(x[i],w) + w0
        y_temp = y[i]
        Grad0 += y_temp * sigmoide(-y_temp*IntoSigma[0][i])
    Grad[0] = - (1/n) * Grad0

    for j in range(1, p+1) :
        Gradj = 0
        for i in range(n) :
            IntoSigma[j][i] = np.dot(x[i],w) + w0
            y_temp = y[i]
            Gradj += y_temp * x[i][j-1]* sigmoide(-y_temp*IntoSigma[j][i])
        Grad[j] = ( - (1/n) * Gradj ) + rho * w[j-1]
    
    Hess00 = 0
    for i in range(n) :
        Hess00 += sigmoide(-y[i]*IntoSigma[0][i])
    Hess[0][0] = (1/n) * Hess00

    for l in range(1, p+1) :
        for k in range(l, p+1) :
            Hesslk = 0
            for i in range(n) :
                Hesslk += x[i][l-1] * x[i][k-1] * sigmoide(-y[i]*IntoSigma[l][i])
            if (l == k) :
                Hess[l][k] = ( (1/n) * Hesslk ) + rho
            else :
                Hess[l][k] = (1/n) * Hesslk
    
    return (Grad, Hess)


def f1_function(w0,w) :
    function = 0
    for i in range(n) :
        function += np.log(1+np.exp(-y[i]*np.dot(x[i],w)+w0)) + (rho/2)*(np.linalg.norm(w))**2
    return (function/n)

def f1_grad(w0,w) :
    Grad = np.zeros(p+1)
    Grad0 = 0
    for i in range(n) :
        y_temp = y[i]
        Grad0 += y_temp * sigmoide(-y_temp*(np.dot(x[i],w) + w0))
    Grad[0] = - (1/n) * Grad0
    for j in range(1, p+1) :
        Gradj = 0
        for i in range(n) :
            y_temp = y[i]
            Gradj += y_temp * x[i][j-1]* sigmoide(-y_temp*(np.dot(x[i],w) + w0))
        Grad[j] = ( - (1/n) * Gradj ) + rho * w[j-1]
    return (Grad)

def f1_hess(w0,w) :
    Hess = np.zeros((p+1,p+1))
    Hess00 = 0
    for i in range(n) :
        Hess00 += sigmoide(-y[i]*(np.dot(x[i],w) + w0))
    Hess[0][0] = (1/n) * Hess00

    for l in range(1, p+1) :
        for k in range(l, p+1) :
            Hesslk = 0
            for i in range(n) :
                Hesslk += x[i][l-1] * x[i][k-1] * sigmoide(-y[i]*(np.dot(x[i],w) + w0))
            if (l == k) :
                Hess[l][k] = ( (1/n) * Hesslk ) + rho
            else :
                Hess[l][k] = (1/n) * Hesslk
    return (Hess)

In [23]:
#print(deriv_f1(1,np.ones(p)))
#print(f1_function(1,np.ones(p)))
#print(f1_grad(1,np.ones(p)))
w0, w = np.ones(1),np.ones(p)
check_grad(f1_function, f1_grad, w0, w)
#print(f1_hess(1,np.ones(p)))

2.1823083731079915

### Question 3.3

In [21]:
epsilon = 1e-10

def newton(w0,w) :
    param0 = w0
    params = w
    g = f1_grad(w0,w)
    grads = []
    while np.dot(g,g) >= epsilon :
        g = f1_grad(param0,params)
        grads.append(g)
        Complex = - solve(f1_HESS(np.array([param0, params])), g(np.array([param0, params])))
        param0 += Complex[0]
        params += Complex[1:]
    return (grads)

def f1_HESS(w):
    return f1_hess(w[0], w[1:])

def f1_GRAD(w) :
    return f1_grad(w[0], w[1:])

In [ ]:
w0, w = np.zeros(1), np.zeros(p)
print(newton(w0,w))

### Question 3.4

In [ ]:
w0, w = (np.ones(1), np.ones(p))
print(newton(w0,w))

### Question 3.5

In [22]:
def Armijo(w0, w, dir, alpha) :
    g = f1_grad(w0,w)
    nor = np.dot(g,g.T)
    while (f1_function((w0,w) + dir*alpha) > f1_function(w0,w) + alpha * nor) :
        alpha = alpha / 2
    return (alpha)

In [ ]:
direction = f1_grad(w0,w)
alph = 0.5
print(Armijo(w0,w,direction,alph))

### Question 4.1

We can't use Newton's method because $\left\|w\right\|_1$ is not differentiable.

### Question 4.2

Obviously, we obtain :
$$
f_2(w_0,w) = \frac{1}{n} \sum_{i=1}^{n} \log(1+\exp(-y_i (x_i^Tw+w_0)))
$$
$$
g_2(w) = \frac{\rho}{2}  \left\|w\right\|_1^2
$$
We have that :
$$
prox_{g_{2}}(x) = \argmin _y ( g_2(y) + \left\|x-y\right\|_2^2 )
$$

In [25]:
def f2_grad(w0,w) :
    Grad = np.zeros(p+1)
    Grad0 = 0
    for i in range(n) :
        y_temp = y[i]
        Grad0 += y_temp * sigmoide(-y_temp*(np.dot(x[i],w) + w0))
    Grad[0] = - (1/n) * Grad0
    for j in range(1, p+1) :
        Gradj = 0
        for i in range(n) :
            y_temp = y[i]
            Gradj += y_temp * x[i][j-1]* sigmoide(-y_temp*(np.dot(x[i],w) + w0))
        Grad[j] = ( - (1/n) * Gradj )
    return (Grad)

### Question 4.3